# Dynamically route logic based on input

This notebook covers how to do routing in the LangChain Expression Language.

Routing allows you to create non-deterministic chains where the output of a previous step defines the next step. Routing helps provide structure and consistency around interactions with LLMs.

There are two ways to perform routing:

    Using a RunnableBranch.
    Writing custom factory function that takes the input of a previous step and returns a runnable. Importantly, this should return a runnable and NOT actually execute.

We’ll illustrate both methods using a two step sequence where the first step classifies an input question as being about LangChain, Anthropic, or Other, then routes to a corresponding prompt chain.

# Using a RunnableBranch

A RunnableBranch is initialized with a list of (condition, runnable) pairs and a default runnable. It selects which branch by passing each condition the input it’s invoked with. It selects the first condition to evaluate to True, and runs the corresponding runnable to that condition with the input.

If no provided conditions match, it runs the default runnable.

Here’s an example of what it looks like in action:

In [2]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
model_config_path = os.path.abspath(os.path.join('../custom_llms/'))
sys.path.insert(0, module_path)
sys.path.insert(0, model_config_path)

from custom_llms import (
    ZhipuAIEmbeddings,
    Zhipuai_LLM,
    load_api
)
api_key = load_api()
model = Zhipuai_LLM(zhipuai_api_key=api_key)

In [3]:
# from langchain.chat_models import ChatAnthropic
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

First, let’s create a chain that will identify incoming questions as being about LangChain, Anthropic, or Other:

In [4]:
chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | model
    | StrOutputParser()
)

In [5]:
chain.invoke({"question": "how do I call Anthropic?"})

'Anthropic'

Now, let’s create three sub chains:

In [6]:
langchain_chain = (
    PromptTemplate.from_template(
        """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | model
)
anthropic_chain = (
    PromptTemplate.from_template(
        """You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | model
)
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question:

Question: {question}
Answer:"""
    )
    | model
)

In [7]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,
)

In [8]:
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch

In [9]:
full_chain.invoke({"question": "how do I use Anthropic?"})

'As Dario Amodei told me, Anthropic is a company that specializes in creating AI systems designed to work in collaboration with humans. To use Anthropic\'s services, you\'ll need to follow these steps:\\n\\n1. Understand your AI requirements: Before approaching Anthropic, identify the specific challenges you want to address and the type of AI solution you\'re looking for. Consider the domain, the problem you want to solve, and the desired outcomes.\\n2. Visit the Anthropic website: Visit the Anthropic website (<https://www.anthropic.com/>) to learn more about their mission, services, and recent projects. This will give you a better understanding of their expertise and capabilities.\\n3. Contact Anthropic: Reach out to their team through the \\"Contact\\" section of their website or via email at [hello@anthropic.com](mailto:hello@anthropic.com). Describe your project and discuss your AI requirements with them.\\n4. Discuss collaboration possibilities: Anthropic\'s team will assess your 

In [10]:
full_chain.invoke({"question": "how do I use LangChain?"})

"As Harrison Chase told me, LangChain is a decentralized platform that enables users to conduct cross-border transactions in various cryptocurrencies and fiat currencies. To use LangChain, follow these steps:\\n\\n1. Set up a LangChain wallet: Visit the LangChain website (https://langchain.io/) and create an account by signing up with your email address. Once you've verified your email, you'll have access to your LangChain wallet.\\n\\n2. Choose a language: LangChain supports multiple languages, so you can choose the one that suits you best. This will help you navigate the platform and understand its features more easily.\\n\\n3. Fund your wallet: You can fund your LangChain wallet using various cryptocurrencies or fiat currencies. Depending on your region, you may be able to use popular payment methods like bank transfers, credit/debit cards, or PayPal. Ensure that you follow the proper guidelines to deposit funds into your wallet.\\n\\n4. Learn about the ecosystem: Familiarize yourse

In [11]:
full_chain.invoke({"question": "whats 2 + 2"})

'Two plus two equals four.'

## Using a custom function
You can also use a custom function to route between different outputs. Here’s an example:

In [12]:
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

In [13]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)

In [14]:
full_chain.invoke({"question": "how do I use Anthropic?"})

"As Dario Amodei told me, Anthropic is a company that specializes in creating AI systems that are designed to work in collaboration with humans. To use Anthropic's technology, you can follow these steps:\\n\\n1. Identify your needs: Determine the specific problems or tasks you want to solve using AI. Anthropic's technology can be applied to a wide range of applications, such as natural language processing, computer vision, and recommendation systems.\\n2. Choose the right Anthropic product: Explore Anthropic's product offerings to find the one that best fits your needs. Their platform provides a comprehensive suite of AI tools and services, including Apify, a low-code platform for building and deploying AI applications, and Apigency, a solution for managing and scaling API-driven systems.\\n3. Integrate Anthropic's API: Once you've selected a product, you'll need to integrate it with your existing systems. Anthropic provides detailed documentation and support to help you integrate thei

In [15]:
full_chain.invoke({"question": "how do I use LangChain?"})

"As Harrison Chase told me, LangChain is a platform that enables users to interact with multiple blockchain networks and digital assets in a seamless and secure manner. To use LangChain, follow these steps:\\n\\n1. Set up your account: Visit the LangChain website (https://langchain.io/) and create an account by signing up with your email address. Verify your account through the email sent to you.\\n\\n2. Choose a wallet: LangChain supports multiple wallets like MetaMask, Trust Wallet, and others. Choose the wallet that best suits your needs and install the appropriate extension or app on your browser or mobile device.\\n\\n3. Connect your wallet: After setting up your LangChain account, connect your chosen wallet to your LangChain profile. This allows you to interact with the blockchain networks and digital assets you have access to.\\n\\n4. Explore the dashboard: Log in to your LangChain account and explore the dashboard. You can view your balance, transactions, and access various fea

In [16]:
full_chain.invoke({"question": "whats 2 + 2"})

'Two plus two equals four.'